In [1]:
!pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score



In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"FACT_CUSTOMER_ORDERS"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [4]:
data = df[['PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [5]:
data['ORDER_DATE']=pd.to_datetime(data['ORDER_DATE'])

In [6]:
data_grouped = data.groupby(['PRODUCT_ID',data['ORDER_DATE'].dt.to_period('M')]).agg(Total_Order_Qty=('ORDER_QTY','sum')).reset_index()

In [7]:
data_grouped['ORDER_MONTH']=data_grouped['ORDER_DATE'].dt.to_timestamp()

In [9]:
data_grouped = data_grouped[['PRODUCT_ID','ORDER_MONTH','Total_Order_Qty']]

In [10]:
data_grouped

,PRODUCT_ID,ORDER_MONTH,Total_Order_Qty
0,Product_001,2019-01-01,148400
1,Product_001,2019-02-01,146650
2,Product_001,2019-03-01,133150
3,Product_001,2019-04-01,153600
4,Product_001,2019-05-01,154650
...,...,...,...
2987,Product_100,2024-04-01,30200
2988,Product_100,2024-05-01,27850
2989,Product_100,2024-06-01,33200
2990,Product_100,2024-07-01,33300


In [11]:
data_grouped = data_grouped.set_index('ORDER_MONTH')

In [12]:
data_grouped

,PRODUCT_ID,Total_Order_Qty
ORDER_MONTH,,
2019-01-01,Product_001,148400
2019-02-01,Product_001,146650
2019-03-01,Product_001,133150
2019-04-01,Product_001,153600
2019-05-01,Product_001,154650
...,...,...
2024-04-01,Product_100,30200
2024-05-01,Product_100,27850
2024-06-01,Product_100,33200


In [13]:
data_grouped.head(1000)

,PRODUCT_ID,Total_Order_Qty
ORDER_MONTH,,
2019-01-01,Product_001,148400
2019-02-01,Product_001,146650
2019-03-01,Product_001,133150
2019-04-01,Product_001,153600
2019-05-01,Product_001,154650
...,...,...
2022-08-01,Product_061,26550
2022-09-01,Product_061,21550
2022-10-01,Product_061,31500


In [14]:
data_grouped.head(50)

,PRODUCT_ID,Total_Order_Qty
ORDER_MONTH,,
2019-01-01,Product_001,148400
2019-02-01,Product_001,146650
2019-03-01,Product_001,133150
2019-04-01,Product_001,153600
2019-05-01,Product_001,154650
2019-06-01,Product_001,142600
2019-07-01,Product_001,142800
2019-08-01,Product_001,127750
2019-09-01,Product_001,120550
